Apply synthetic control to see if the trend in area burned evolved differently in the independent regions who kept the Corpo Forestale, compared to all other regions where since 2017 the Carabinieri Forestali are the responsible authority.

Data for area burned since 2008: https://docs.google.com/spreadsheets/d/1Znp3QEZqN_mNEcS6cK9d0Laa4zLtUp6wa08aCQtwZU0/edit#gid=1359845726 
Provinces of the autonomous regions are:
Friuli-Venezia-Giulia: Udine, Pordenone, Gorizia, Trieste
Sardinia: Sassari, Nuoro, Cagliari, Oristano, Olbia-Tempio, Ogliastra, Medio Campidano, Carbonia-Iglesias
Sicily: Palermo, Trapani, Agrigento, Caltanissetta, Enna, Ragusa, Siracusa, Catania, Messina 
Trentino-Alto Adige/Südtirol: Trento, Bolzano
Val d’Aosta: Valle d'Aosta/Vallée d'Aoste


In 2017, the Corpo Forestale dello Stato (forestry and prevention approach to wildfires) in each region was scrapped and replaced by the Carabinieri Forestali (civil protection/emergency/response approach). The autonomous regions however kept the Corpo Forestale dello Stato (Friuli-Venezia Giulia, Sardinia, Sicily, Trentino-Alto Adige/Südtirol, and Val d'Aosta)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
# Import packages.
import cvxpy as cp
import numpy as np

In [ ]:
# df = pd.read_excel('../data/NUTS2021.xlsx', sheet_name='Italia')
# df = df.rename(columns={'NUTS level 2' : 'Region', 'NUTS level 3' : 'Province'})
# df = df[['Region', 'Province']]
# df['Region'] = df['Region'].fillna(method='pad')
# df = df.drop([0, 1])
# df = df.dropna()
# df.to_csv('../data/italy_regions.csv', index=False)

In [ ]:
df_regions = pd.read_csv('../data/italy_regions.csv')
map_regions = {row['Province'] : row['Region'] for _, row in df_regions.iterrows()}

In [ ]:
df = pd.read_csv('../data/Mach l and fire data - Italy - area burned.csv')
df['FIREDATE'] = pd.to_datetime(df['FIREDATE'])
# df['YEAR'] = df['FIREDATE'].dt.year

In [ ]:
df = df.rename(columns={'YEARSEASON' : 'YEAR'})
df = df[df['YEAR'] < 2022]
df['REGION'] = df['PROVINCE'].map(map_regions)
df

In [ ]:
df['AREA_HA'].hist(bins=np.logspace(np.log10(1),np.log10(20000), 20)).set_xscale("log")
plt.gca().set_xlabel('Fire Size')
plt.gca().set_ylabel('Num')

In [ ]:
df['LARGE'] = df['AREA_HA'] >= 100.

In [ ]:
print("COUNTRY:", df['COUNTRY'].unique())
print("PROVINCE:", df['PROVINCE'].unique())

In [ ]:
g = df.groupby(['YEAR', 'PROVINCE'])
g['AREA_HA']

In [ ]:
group_by = 'PROVINCE'
group_by = 'REGION'


# compute metrics
g = df.groupby(['YEAR', 'REGION'])
df_area = g['AREA_HA'].agg('sum')
df_count = g['AREA_HA'].agg('count').rename('NUM_FIRES')
df_count_large = g['LARGE'].agg('sum').rename('NUM_FIRES_LARGE')

df_targets = pd.concat([df_area, df_count, df_count_large], axis=1).reset_index()
if group_by == 'PROVINCE':
    df_targets.insert(2, column='REGION', value='')
    df_targets['REGION'] = df_targets['PROVINCE'].map(map_regions)
df_targets

In [ ]:
# df_targets['NUM_FIRES'].hist()
df_targets['NUM_FIRES_LARGE'].hist(bins=np.logspace(np.log10(1),np.log10(20000), 20)).set_xscale("log")

In [ ]:
df_targets['PROVINCE'].unique().tolist()

In [ ]:
df_targets['REGION'].unique()
df_regions['Region'].unique()

In [ ]:
# Autonomous regions
REGIONS_AUTONOMOUS = [
    'Friuli-Venezia Giulia', 
    'Sardegna',
    'Sicilia',
    "Valle d'Aosta/Vallée d'Aoste",
    "Provincia Autonoma di Bolzano/Bozen",
]

REGIONS_OTHERS = [r for r in df_targets['REGION'].unique() if not r in REGIONS_AUTONOMOUS]


In [ ]:
# plot total area burned
fig, ax = plt.subplots(figsize=(10,4))
for key, grp in df_targets.groupby('PROVINCE'):
    if key in PROVINCES_SICILY:
        ax.plot(grp['YEAR'], grp['AREA_HA'].cumsum(), label=key)

ax.legend()
ax.set_title("TOTAL AREA BURNED SICILY")
ax.set_ylabel("AREA HA")
ax.set_xlabel("YEAR")
plt.show()

In [ ]:
# plot total number of fires
fig, ax = plt.subplots(figsize=(10,4))
for key, grp in df_targets.groupby('PROVINCE'):
    if key in PROVINCES_SICILY:
        ax.plot(grp['YEAR'], grp['NUM_FIRES'].cumsum(), label=key)

ax.legend()
ax.set_title("TOTAL NUM FIRES SICILY")
ax.set_ylabel("COUNT")
ax.set_xlabel("YEAR")
plt.show()

In [ ]:
def synthetic_control(X0, Y0, X1, verbose=False):


#     if log:
#         X0 = np.log(1 + X0)
#         Y0 = np.log(1 + Y0)
#         X1 = np.log(1 + X1)
        
#     norm_x0 = np.mean(X0, axis=0)
#     norm_y0 = np.mean(Y0, axis=0)
   
    t, d = X0.shape
    _, n = Y0.shape
    w = cp.Variable((d, n))
    cost = cp.sum_squares(X0 @ w - Y0)
    prob = cp.Problem(cp.Minimize(cost), 
                      [cp.sum(w, axis=0) == np.ones(n), w >= 0]
                     )
    prob.solve(verbose=verbose, max_iter=100000)
    
#     w = np.ones(d)/d
#     for t in range(10000):
#         grad = 2*(X0 @ w -Y0) @ X0
#         i = np.argmin(grad)
#         w1 = np.zeros(d)
#         w1[i] = 1.
#         gamma = 1/(t + 2)
#         w = (1 - gamma) * w + gamma*w1
    
    
#     # Print result.
    if verbose:
        print("\nThe optimal value is", prob.value)
        print("The optimal x is")
        print(w.value)
        print("The norm of the residual is ", cp.norm((X0 @ w - Y0)/norm, p=2).value)
    w = w.value
    Y1_synthetic = X1 @ w
    Y0_synthetic = X0 @ w
#     print(Y1_synthetic)
#     if log:
#         Y1_synthetic = np.exp(Y1_synthetic) - 1
#         Y0_synthetic = np.exp(Y0_synthetic) - 1
    return w, Y1_synthetic, Y0_synthetic

In [ ]:
target = 'AREA_HA'
# target = 'NUM_FIRES'
# target = 'NUM_FIRES_LARGE'
t0 = 2015


df_treated

In [ ]:
target = 'AREA_HA'
# target = 'NUM_FIRES'
# target = 'NUM_FIRES_LARGE'
t0 = 2015

# df_treated = df_targets[df_targets['REGION'].isin(REGIONS_AUTONOMOUS)].pivot_table(target, ['YEAR',], group_by).sort_index()
# df_control = df_targets[df_targets['REGION'].isin(REGIONS_OTHERS)].pivot_table(target, ['YEAR',], group_by).sort_index()
# df_control = df_control.drop(columns=['Lombardia', 'Emilia-Romagna' , 'Provincia Autonoma di Trento'])

df_treated = df_targets[df_targets['REGION'].isin(REGIONS_OTHERS)].pivot_table(target, ['YEAR',], group_by).sort_index()
df_control = df_targets[df_targets['REGION'].isin(REGIONS_AUTONOMOUS)].pivot_table(target, ['YEAR',], group_by).sort_index()
df_treated = df_treated.drop(columns=['Lombardia', 'Emilia-Romagna' , 'Provincia Autonoma di Trento'])


# drop_cols = ((~df_provinces.isna()).sum(axis=0) < 2) | ((~df_provinces.loc[:t0 - 1].isna()).sum(axis=0)  ==0) | ((~df_provinces.loc[t0:].isna()).sum(axis=0) == 0)
# drop_cols = drop_cols.index[drop_cols].values


# df_provinces = df_provinces.fillna(0)
# df_treated = df_provinces[[p for p in PROVINCES_AUTONOMOUS if not p in (drop_cols.tolist() + ['Sassari'])]].copy()



# # df_treated = df_provinces[PROVINCES_SARDEGNA].copy()
# # df_treated = df_provinces[PROVINCES_SICILY].copy()
# # df_treated = pd.DataFrame(df_treated.sum(axis=1).rename("Sicily"))

# df_control = df_provinces[[p for p in PROVINCES_OTHER if not p in drop_cols.tolist()]]
df_treated = df_treated.fillna(0.)
df_control = df_control.fillna(0.)

df_treated = df_treated.cumsum()
df_control = df_control.cumsum()

df_treated

df_control

In [ ]:
T = df_treated.index
T0 = df_treated.index[df_treated.index < t0]
T1 = df_treated.index[df_treated.index >= t0]
Y0 = df_treated[df_treated.index < t0].values
Y1 = df_treated[df_treated.index >= t0].values
X0 = df_control[df_control.index < t0].values
X1 = df_control[df_control.index >= t0].values


In [ ]:
X0.shape, X1.shape, Y0.shape, Y1.shape

In [ ]:
# _X0 = np.log(1 + X0)
# mean_x0 = np.max(_X0, axis=0)
# _X0 = _X0 - mean_x0
# _X1 = np.log(1 + X1)- mean_x0
# _Y0 = np.log(1 + Y0)
# mean_y0 = np.max(_Y0, axis=0)
# _Y0 = _Y0 - mean_y0


In [ ]:
norm_x0 = X0[-1]
norm_y0 = Y0[-1]
_X0 = X0 / norm_x0
_X1 = X1 / norm_x0
_Y0 = Y0 / norm_y0

In [ ]:
W, Y1_synthetic, Y0_synthetic = synthetic_control(_X0, _Y0, _X1,verbose=False)

# Y1_synthetic = np.exp(Y1_synthetic + mean_y0) - 1
# Y0_synthetic = np.exp(Y0_synthetic + mean_y0) - 1


Y1_synthetic = Y1_synthetic * norm_y0
Y0_synthetic = Y0_synthetic * norm_y0

print(W.shape)
threshold = 0.02

for unit, w in zip(df_treated.columns.values, W.T):
    mask = tuple([w >= threshold])
    print(f'{unit}:' +', '.join([f"({r}, {wi:.2f})" for (r, wi) in zip(df_control.columns.values[mask], w[mask])]))

In [ ]:
cumsum = False
for unit, y0, y1, y1_synethtic, y0_synthetic in zip(df_treated.columns, Y0.T, Y1.T, Y1_synthetic.T, Y0_synthetic.T):
    
    data_real = np.concatenate([y0, y1])
    data_synthetic = np.concatenate([y0, y1_synethtic])
    data_fit  = np.concatenate([y0_synthetic, y1_synethtic])
    if cumsum:
        data_real = data_real.cumsum()
        data_synthetic = data_real.cumsum()
        data_fit = data_fit.cumsum()

    line, = plt.plot(T, data_real, label=unit)
    plt.plot(T, data_synthetic, linestyle='--', color=line.get_color())
    plt.plot(T, data_fit, linestyle=':', color=line.get_color())


plt.axvline(t0 - 1, c='k', linestyle=':')

plt.ylabel(target)
plt.xlabel('YEAR')
plt.legend()

In [ ]:

data_real = np.concatenate([Y0.sum(axis=1), Y1.sum(axis=1)])
data_fit = np.concatenate([Y0_synthetic.sum(axis=1), Y1_synthetic.sum(axis=1)])
data_synthetic = np.concatenate([Y0.sum(axis=1), Y1_synthetic.sum(axis=1)])

if cumsum:
    data_real = data_real.cumsum()
    data_fit = data_fit.cumsum()
    data_synthetic = data_synthetic.cumsum()
    
line, = plt.plot(T, data_real, label='Combined Control')
plt.plot(T, data_fit, linestyle=':', color=line.get_color())
plt.plot(T, data_synthetic, linestyle='--', color=line.get_color())

plt.axvline(t0 - 1, c='k', linestyle=':')

plt.ylabel(target)
plt.xlabel('YEAR')
plt.legend()

TODO:

* Get additional features: Fire index, Weather, Rain, Temps
* Ensure we have good predictions in the pre-treated period. Is there a bug?
* Read synthetic control paper

In [ ]:
df_gfis = pd.read_csv('../data/MCD64A1_burned_area_full_dataset_2002-2019.csv')
df_gfis = df_gfis[df_gfis['Country'] == 'Italy'].copy()

In [ ]:
df_gfis['Total_hectares'] = df_gfis[['Cropland_BA_hectares', 
                                     'Forest_BA_hectares', 
                                     'Grass_and_Shrubland_BA_hectares',
                                     'Wetlands_BA_hectares',
                                     'Settlement_BA_hectares',
                                     'Other_BA_hectares']].sum(axis=1)

df_grouped = df_gfis.groupby(['Year', 'Region'])
df_gfis_agg = df_grouped['Total_hectares'].agg('sum')
df_gfis_agg = df_gfis_agg.reset_index()
df_gfis_agg

In [ ]:
df_gfis_agg = df_gfis_agg[df_gfis_agg['Region'] == 'Sicily']

In [ ]:
df_sicilia = df_targets[df_targets['REGION'] == 'Sicilia'].set_index('YEAR')
df_sicilia

In [ ]:
df_gfis_agg = df_gfis_agg.set_index('Year')
df_gfis_agg

In [ ]:
plt.plot(df_gfis_agg.index, df_gfis_agg['Total_hectares'], label='gwis')
plt.plot(df_sicilia.index, df_sicilia['AREA_HA'], label='Mach l and fire data')
plt.legend()
plt.xticks(np.arange(2002, 2021, 2))
